In [41]:
import pandas as pd
import scipy.io as sio
import numpy as np
import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import re

downRiverDF = pd.read_csv('downriver_2012_2013_processed.csv', parse_dates = ['DN'])
minilanderDF = pd.read_csv('minilander_2012_2013_processed.csv', parse_dates = ['DN'])

#create day DN to separate out data values by day instead of 5 min intervals
downRiverDF['dayDN'] = downRiverDF['DN'].dt.floor('D')
minilanderDF['dayDN'] = minilanderDF['DN'].dt.floor('D')
#takes highest sal values by day and collects idx, then converts floats to integers
idxMaxDownArray = downRiverDF.groupby("dayDN")["BottomSalCorrected"].idxmax().ffill().astype('Int64').to_numpy()
idxMaxMiniArray = minilanderDF.groupby("dayDN")["Salinity"].idxmax().ffill().bfill().astype('Int64').to_numpy()

#filter data frame from NAN duplicates created in previous step
salToTempDownDF = downRiverDF.loc[idxMaxDownArray].drop_duplicates()
salToTempMiniDF = minilanderDF.loc[idxMaxMiniArray].drop_duplicates()

#trying to combine so that both arrays are same length
combinedDF = pd.merge_asof(salToTempDownDF, salToTempMiniDF, on = 'DN')[["DN", "BottomTemp", "Temperature"]]
maxDN = combinedDF["DN"].max()
salToTempMiniDF["BottomTemp"] = np.nan
#combining tail of minilander data to other data
combinedDF = pd.concat([combinedDF, salToTempMiniDF.loc[salToTempMiniDF['DN']>maxDN][["DN", "BottomTemp", "Temperature"]]])

#plot ocean temperature
tempFigLabels = ['BottomTempAtMaxSal', 'MiniTempAtMaxSal']
tempfig = px.line(x = combinedDF['DN'], y = [combinedDF["BottomTemp"], combinedDF["Temperature"]], color_discrete_sequence= ["black", "purple"], title = "Temperature")
for idx in range(len(tempFigLabels)):
    tempfig.data[idx].name = tempFigLabels[idx]
    tempfig.data[idx].hovertemplate = 'variable=' + tempFigLabels[idx] + '<br>x=%{x}<br>value=%{y}<extra></extra>'
    tempfig.data[idx].legendgroup = tempFigLabels[idx]

tempfig.update_layout(title=dict(text= "Ocean Temperature for 2012-2013", font=dict(size=25)))
tempfig.update_xaxes(tickangle=30)
tempfig.update_xaxes(rangeslider_visible=True)
tempfig.update_xaxes(range = [pd.Timestamp('2012-12-10'),pd.Timestamp('2013-05-26')])

tempfig.show() 
